In [1]:
# Install all dependencies
!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install PyPDF2

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 656.4 kB/s eta 0:00:37
   ---------------------------------------- 0.2/24.0 MB 1.3 MB/s eta 0:00:19
    --------------------------------------- 0.4/24.0 MB 2.3 MB/s eta 0:00:11
   - -------------------------------------- 0.8/24.0 MB 3.5 MB/s eta 0:00:07
   - -------------------------------------- 1.0/24.0 MB 4.1 MB/s eta 0:00:06
   - -------------------------------------- 1.0/24.0 MB 4.1 MB/s eta 0:00:06
   - -------------------------------------- 1.0/24.0 MB 4.1 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/24.0 MB 3.2 MB/s eta 0:00:08
   --- ------------------------------------ 1.9/24.0 MB 4.1 MB/s eta 0:00:06
   ---- ----------------------------------- 2.4/24.0 MB 4.8 MB/s eta 0:00:05
   ---- ----------------------------------- 3.0/24.0 MB 5.4 MB/s eta 0:00:04
   ----- --

In [4]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ARITRA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
# Load data
df = pd.read_csv('nyc-jobs-1.csv')
# Check data
df.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,,,...,"Salary range for this position is: $42,405 - $...",,,,NaN,New York City residency is generally required ...,2011-06-24T00:00:00,NaN,2011-06-24T00:00:00,2018-07-17T00:00:00
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,,F,...,,"In addition to applying through this website, ...",,,NaN,New York City residency is generally required ...,2012-01-26T00:00:00,NaN,2012-01-26T00:00:00,2018-07-17T00:00:00
2,102221,DEPT OF ENVIRONMENT PROTECTION,External,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
3,102221,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
4,114352,DEPT OF ENVIRONMENT PROTECTION,Internal,5,Deputy Plant Chief,SENIOR STATIONARY ENGINEER (EL,91639,0,,F,...,Appointments are subject to OMB approval Fo...,"Click ""Apply Now"" button",40 per week / day,Various,NaN,New York City residency is generally required ...,2012-12-12T00:00:00,NaN,2012-12-13T00:00:00,2018-07-17T00:00:00


In [6]:
# Show column name
df.columns

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Code No', 'Level', 'Job Category',
       'Full-Time/Part-Time indicator', 'Salary Range From', 'Salary Range To',
       'Salary Frequency', 'Work Location', 'Division/Work Unit',
       'Job Description', 'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')

In [7]:
df =df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']]
df.head()

,Business Title,Job Description,Minimum Qual Requirements,Preferred Skills
0,Account Manager,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,â€¢\tExcellent interpersonal and organizationa...
1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,
2,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,
3,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,
4,Deputy Plant Chief,"Under general direction, is in responsible cha...",1. Six years of full-time satisfactory experie...,


In [8]:
# Create a new column called 'data' and merge the values of the other columns into it
df['data'] = df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
df.drop(['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills'], axis=1, inplace=True)
# Preview the updated dataframe
print(df.head())

                                                data
0  Account Manager Division of Economic & Financi...
1  EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT The N...
2  Project Specialist Under direct supervision, p...
3  Project Specialist Under direct supervision, p...
4  Deputy Plant Chief Under general direction, is...


In [10]:
# Tag data
data = list(df['data'])
tagged_data = [TaggedDocument(words = word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

In [15]:
import multiprocessing

In [16]:
multiprocessing.cpu_count()

12

In [21]:
# Model initialization
import multiprocessing
model = Doc2Vec(vector_size = 50,
                min_count = 5,
                epochs = 100,
                alpha = 0.001,
                workers=multiprocessing.cpu_count()
)

In [22]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

8599


In [23]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
    model.train(tagged_data, 
                total_examples=model.corpus_count,
                epochs=model.epochs)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/100
Training epoch 2/100
Training epoch 3/100
Training epoch 4/100
Training epoch 5/100
Training epoch 6/100
Training epoch 7/100
Training epoch 8/100
Training epoch 9/100
Training epoch 10/100
Training epoch 11/100
Training epoch 12/100
Training epoch 13/100
Training epoch 14/100
Training epoch 15/100
Training epoch 16/100
Training epoch 17/100
Training epoch 18/100


KeyboardInterrupt: 

In [65]:
# import OS module
import os
# Get the list of all files and directories
path = r"C:\Users\ARITRA\Documents\Notebooks\Project Recommender Hackathon Trumio\Resumes"
dir_list = os.listdir(path)
dir_list

['Abhishek Kumar Tiwari.pdf',
 'Aditya Chandrikapure.pdf',
 'Aditya Jha.pdf',
 'Aditya Krishna Das.pdf',
 'Akshat Vijay.pdf',
 'Aman Mishra.pdf',
 'Amisha Sinha.pdf',
 'Amit Kumar.pdf',
 'Amruit Sahoo.pdf',
 'Anirudh Kohli.pdf',
 'Anirudh Sharma.pdf',
 'Anjali Raj.pdf',
 'Ankesh Kumar.pdf',
 'Anurag Anand.pdf',
 'Apoorva Tejaswi.pdf',
 'Arihant Shukla.pdf',
 'Aryan Kumar.pdf',
 'Aryan Tayal.pdf',
 'Atul Ranjan.pdf',
 'Bharat Kabra.pdf',
 'Bhaskar Sugana.pdf',
 'Chirag Agarwal.pdf',
 'Dibyendu Biswas.pdf',
 'Dnyandeep Chavan.pdf',
 'Dokala Avinash.pdf',
 'Doraswamy Vamsi.pdf']

In [67]:
resumes = []
for name in dir_list:
    file_path = f"Resumes\{name}"
    pdf = PyPDF2.PdfReader(file_path)
    resume = ""
    for i in range(len(pdf.pages)):
        pageObj = pdf.pages[i]
        resume += pageObj.extract_text()
    resumes.append(resume)

In [72]:
jd1 = """
Hiring - Chatbot
You are designing a chatbot that returns the best engineers for a given set of instructions
provided to the chatbot. The chatbot should interface with the user and return results
dynamically based on the information provided.
Information you should support for the search:
● Whether the worker is full-time or part-time
● Whether the worker fits the user’s budget
● Whether the worker has the skills that the user wants (i.e. Python, React, AWS)
● Other queries in natural language (i.e. “worked at a big tech company”)
Example queries:
● “I want to hire someone with experience in Python and Node. My budget is $10000 a
month.”
○ The chatbot follows up asking whether the user wants a full-time or part-time
worker after showing some results
● “I want to hire someone who worked at a big tech company. I have an unlimited budget.
They should be proficient in Python.”
○ The chatbot follows up asking whether the user wants a full-time or part-time
worker after showing some results
● “I want to hire a developer”
○ The chatbot follows up asking for the skills, budget, and whether the worker is
part-time or full-time. The chatbot shows results after skills are provided
This search has two components: a scalar part (part-time / full-time, budget, and skills, which
are all stored in the database) and a semantic part (supporting all other queries in natural
language, which will require embeddings and a vector database).
Your search should be as low latency as possible. You will likely need to utilise various DB tricks
to make this happen.
You should also build out a front-end to make testing this chatbot easier.
Additionally, your search should still work properly if users from the database are deleted or if
their resume information is mutated (which requires giving some thought to the connection
between Pinecone and the SQL database).
Lastly, you should design a ranking algorithm to prioritize how the candidates are returned in the
search. This should take into account the following:
● Their background (work experience, education, etc.)
● Github data (pre-processed and scored from their Github username). Note: these are not
the actual Github usernames of the individuals, they are randomly selected public Github
usernames.
It is entirely up to you how you implement this ranking and how granular you want to be with
scoring each component of a candidate’s background. It is also up to you how you want to
prioritize each score when returning candidates.
Tips and Recommendations
Tools I’d recommend using for various components of the problem statement:
● Visualizing data in the DB: https://www.mysql.com/products/workbench/
● Stateless, lightweight tasks: https://cloud.google.com/functions?hl=en
● Stateful, heavyweight tasks: https://cloud.google.com/run?hl=en or your own Google VM
● Vector database for the semantic component of the search: https://www.pinecone.io/ or
https://www.weaviate.io or something similar
● Embeddings model and reranker for the semantic search:
https://huggingface.co/BAAI/bge-large-en-v1.5#using-sentence-transformers (you could
also use OpenAI’s text embeddings)
You will likely need to use embeddings and a vector database for the semantic component of
the search. Embeddings are numerical representations of text — for the purposes of this trial,
you should embed the text that should be searchable via natural language (work experience,
education, etc.). Then, you can take the semantic component of the search query, embed it, and
do a cosine similarity to return the most relevant items. You will need to build out a way to
present the data between the scalar elements of the search and the semantic elements.
Tutorial for vector databases: https://www.pinecone.io/learn/vector-database/
If you have any questions, feel free to reach out!
"""

In [73]:
jd2 = """
Show Me The Money
The goal of the project is to build a simple one page application to display the Balance
Sheet Report from Xero.
Please read through the API documentation at
https://developer.xero.com/documentation/api/accounting/reports#balance-sheet.
Use mock Xero Balance Sheet API docker image available at
https://hub.docker.com/r/jaypeng2015/show-me-the-money.
The Server runs on http with port 3000, and the api path is /api.xro/2.0/Reports/BalanceSheet
once you have it running.
The system should consist of the following:
● Backend - Any typed Language (except Java)
○ Assume that the authentication with Xero is already done.
○ Provide API endpoint to get data from Xero API for the frontend to use.
○ Consider error handling.
○ Consider unit tests.
● Frontend - Typescript + React
○ Display the results in a table based on the data structure return from Xero.
○ Consider unit tests.
Consider containerise your solution.
"""

In [74]:
jd3 = """
Hiring - Chatbot
You are designing a chatbot that returns the best engineers for a given set of instructions
provided to the chatbot. The chatbot should interface with the user and return results
dynamically based on the information provided.
Information you should support for the search:
● Whether the worker is full-time or part-time
● Whether the worker fits the user’s budget
● Whether the worker has the skills that the user wants (i.e. Python, React, AWS)
● Other queries in natural language (i.e. “worked at a big tech company”)
Example queries:
● “I want to hire someone with experience in Python and Node. My budget is $10000 a
month.”
○ The chatbot follows up asking whether the user wants a full-time or part-time
worker after showing some results
● “I want to hire someone who worked at a big tech company. I have an unlimited budget.
They should be proficient in Python.”
○ The chatbot follows up asking whether the user wants a full-time or part-time
worker after showing some results
● “I want to hire a developer”
○ The chatbot follows up asking for the skills, budget, and whether the worker is
part-time or full-time. The chatbot shows results after skills are provided
This search has two components: a scalar part (part-time / full-time, budget, and skills, which
are all stored in the database) and a semantic part (supporting all other queries in natural
language, which will require embeddings and a vector database).
Your search should be as low latency as possible. You will likely need to utilise various DB tricks
to make this happen.
You should also build out a front-end to make testing this chatbot easier.
Additionally, your search should still work properly if users from the database are deleted or if
their resume information is mutated (which requires giving some thought to the connection
between Pinecone and the SQL database).
Lastly, you should design a ranking algorithm to prioritize how the candidates are returned in the
search. This should take into account the following:
● Their background (work experience, education, etc.)
● Github data (pre-processed and scored from their Github username). Note: these are not
the actual Github usernames of the individuals, they are randomly selected public Github
usernames.
It is entirely up to you how you implement this ranking and how granular you want to be with
scoring each component of a candidate’s background. It is also up to you how you want to
prioritize each score when returning candidates.
Tips and Recommendations
Tools I’d recommend using for various components of the problem statement:
● Visualizing data in the DB: https://www.mysql.com/products/workbench/
● Stateless, lightweight tasks: https://cloud.google.com/functions?hl=en
● Stateful, heavyweight tasks: https://cloud.google.com/run?hl=en or your own Google VM
● Vector database for the semantic component of the search: https://www.pinecone.io/ or
https://www.weaviate.io or something similar
● Embeddings model and reranker for the semantic search:
https://huggingface.co/BAAI/bge-large-en-v1.5#using-sentence-transformers (you could
also use OpenAI’s text embeddings)
You will likely need to use embeddings and a vector database for the semantic component of
the search. Embeddings are numerical representations of text — for the purposes of this trial,
you should embed the text that should be searchable via natural language (work experience,
education, etc.). Then, you can take the semantic component of the search query, embed it, and
do a cosine similarity to return the most relevant items. You will need to build out a way to
present the data between the scalar elements of the search and the semantic elements.
Tutorial for vector databases: https://www.pinecone.io/learn/vector-database/
If you have any questions, feel free to reach out!
"""

In [75]:
jd4 = """
Web Scraper
Objective: Given that summer holidays have already started, hotels and resorts in the
top tourist destinations of India are already facing high demand. However, properties
have different ways of managing inventories at different platforms. When I as a traveller
go to platform1, a hotel/resort may show as Out of Stock there but another platform still
shows inventory left. For non tech savvy travelers, it might be very hectic and
cumbersome to look for availability on different platforms.
Create a web scraping tool where if given a property, checkin/checkout dates and
number of guests/rooms can give me a list of properties and platforms where inventory
is available on the given dates.
Project Idea: For this web scraping project, first, pick a website serving hotels/property
booking facilities, such as MakeMytrip, Agoda, AirBnB or Booking.com. Feed-in your
details using an automated fashion, and then you can crawl the website to fetch the
ticket price details.
You can suitably use tools like Selenium or Pupeteer for performing web scraping in this
project.
"""

In [76]:
jd5 = """
Problem Statement for Digital Marketing Professional: Website
Visibility & SEO Improvement
Overview:
I have recently launched a hotel booking website built with ReactJs. It has now been a
week since the launch, but unfortunately, the website is not appearing in Google search
results or any other major search engines. This is a critical issue as the success of the
website heavily relies on organic traffic and visibility. I am looking for a digital marketing
professional to conduct an in-depth analysis of the situation, identify the root causes of
these issues, and develop a comprehensive digital marketing plan to improve the
website's visibility and search engine ranking.
Key Objectives:
1. Initial Research and Analysis:
● Conduct a detailed SEO audit of the website to identify any technical SEO
issues.
● Analyze if the website has been indexed by Google. If not, identify the reasons.
● Review the website to ensure it complies with Google's Webmaster Guidelines.
● Verify the presence and accuracy of meta tags, title tags, headers (H1, H2, etc.),
and other critical on-page SEO elements.
● Check for mobile-friendliness and responsiveness of the website.
● Evaluate the website's loading speed and overall performance metrics.
● Review content quality and keyword optimization across the pages.
2. Competitor Analysis:
● Identify key competitors in the hotel booking industry and analyze their digital
marketing strategies.
● Study competitors' keyword rankings, backlink profiles, and content strategies.
● Determine the gaps between my website and competitors' websites in terms of
SEO.
3. Technical SEO:
● Check and fix any issues related to robots.txt file and XML sitemap.
● Ensure proper use of canonical tags to avoid duplicate content issues.
● Analyze and optimize the website's internal linking structure.
● Verify that the website is secure (SSL/TLS).
● Audit the website for schema markup implementation and enhancement
opportunities.
4. Content Strategy:
● Evaluate existing content for relevance, uniqueness, and keyword optimization.
● Develop a content strategy that includes a mix of blog posts, hotel reviews, travel
guides, and other engaging content.
● Plan a keyword research strategy to target short-tail and long-tail keywords.
● Create a content calendar for consistent publishing.
5. User Experience (UX) & Design:
● Assess the website’s design for user-friendliness and ease of navigation.
● Identify and recommend improvements for user engagement metrics (e.g.,
bounce rate, session duration).
"""

In [77]:
jd6 = """
Problem Statement for UX Designers: B2B SaaS Platform for
Restaurant Owners and Suppliers
Overview:
We are developing a pioneering B2B SaaS platform designed to connect restaurant
owners with manufacturers and suppliers. This platform aims to streamline the
procurement process for restaurant owners while providing manufacturers and suppliers
with an efficient avenue to reach their market. Given our diverse user base, which
includes individuals with varied levels of digital literacy, the platform must be extremely
user-friendly, intuitive, and visually appealing.
We need UX designers to create a mobile-first user interface that adapts seamlessly
across different devices and screen sizes. The platform should exhibit a consistent
styling for color schemes, typography, themes, and layouts, aligning with the Tailwind
CSS design system.
Key Objectives:
1. User Research and Personas:
● Conduct comprehensive user research with both restaurant owners and
suppliers to understand their needs, pain points, and behavior patterns.
● Develop detailed user personas to ensure the design meets the specific needs of
these target groups.
2. Information Architecture:
 - Create a logical and intuitive information architecture that facilitates easy navigation
through the platform’s features, such as browsing suppliers, placing orders, and tracking
shipments.
 - Design the structure to accommodate future scalability as the platform grows Key
Objectives (Continued):
3. Wireframes and Prototypes:
- Develop wireframes for all major components and user flows, including supplier
browsing, order placement, order tracking, user profile management, and
communication tools (chat or messaging systems).
- Create interactive prototypes to test usability and gather feedback from potential
users.
- Ensure that the prototypes reflect a mobile-first design approach, while also providing
a seamless experience on tablets and desktops.
4. Visual Design:
- Develop a consistent visual style that aligns with the Tailwind CSS design system,
incorporating a coherent color scheme, typography, and themes.
- Ensure that the design elements are consistent across all pages and functionalities,
creating a unified user experience.
- Choose colors, fonts, and layouts that enhance readability and don’t distract users
from completing key tasks.
5. Responsive Design:
- Implement a mobile-first design strategy, ensuring that all features and user interfaces
are fully functional and visually appealing on smaller screens.
- Design breakpoints and responsive elements to ensure a seamless transition and
uniform experience across various devices and screen sizes.
6. Accessibility:
- Design with accessibility in mind, ensuring that the platform is usable by individuals
with disabilities.
- Ensure compliance with WCAG (Web Content Accessibility Guidelines) standards to
make the platform inclusive for all users.
7. User Flows and Interaction Design:
- Design intuitive user flows that minimize friction and streamline interactions, ensuring
that users can accomplish their goals with minimal steps.
- Focus on key interactions, such as searching for suppliers, filtering products, adding
items to the cart, and checking out, to ensure a smooth user experience.
8. Onboarding and Tutorials:
- Develop an effective onboarding process for new users that introduces them to the
platform’s features and guides them through initial setup.
- Create in-app tutorials and tooltips that assist users in understanding and utilizing the
platform’s functionalities.
9. Feedback and Iteration:
- Implement mechanisms for users to provide feedback on their experience, such as
surveys or in-app feedback forms.
- Use this feedback to iteratively improve the design, making necessary adjustments to
enhance usability and satisfaction.
10. Consistency with Tailwind CSS:
- Ensure that all design components are built with Tailwind CSS, utilizing its utility-first
approach to create a cohesive and maintainable design system.
- Leverage Tailwind's predefined style utilities for rapid development and ensure that
custom styles are minimal and necessary.
"""

In [78]:
jd7 = """
Market Survey
Speechofy is a software which is one of the Best AI text to speech for Chromium
browsers.
Objective: The objective of the project is to speak to potential customers of Speechofy
to understand buying behaviour and position Speechofy software.
These leads could be followed up by the Speechofy sales team to close the sale. The
research on buying behaviour will also enable Speechofy to fine tune its messaging and
sales approach to additional leads.
Description: Understand Speechofy’s business, products and clients and design a
Customer Survey questionnaire. This will be done along with a team at Speechofy.
Use the survey questionnaire and speak to 40 potential customers / leads of Speechofy
to collect information and position Speechofy as a good solution.
Analyse the results of the questionnaire and provide suggestions to Speechofy on how
to better message its product to potential customers.
A team of 2 students who are pursuing a degree in Marketing or Technology (with
knowledge/interest in both technology and marketing) will be the preferred combination
for this project.
Deliverables: The milestone deliverables for this project should be Design questionnaire
(Milestone 1), Conduct customer surveys (Milestone 2), Analyse surveys and provide
recommendations to Speechofy for follow up (Milestone 3)
"""

In [79]:
jd8 = """
Problem Statement for Content Writer and Copywriter: Blog on
Advances in AI and LLMs
Overview:
Artificial Intelligence (AI) and Large Language Models (LLMs) have seen unprecedented
advancements in recent years, transforming industries and everyday life. As these
technologies evolve, they introduce powerful tools that revolutionize how we work,
communicate, and solve problems. However, with rapid progress comes a need for
comprehensive insights to educate the public about these developments, the major
players in the market, the best generative AI software available, and the potential
impacts on society, both positive and negative.
We are seeking a skilled content writer and copywriter to craft a detailed blog post that
explores the recent advancements in AI and LLMs, compares the contributions of
leading companies like OpenAI, Google, and Microsoft, and discusses the safety
concerns and future threats associated with AI.
Key Objectives:
1. Introduction to AI and LLMs:
- Provide an engaging introduction to AI and Large Language Models, explaining what
they are and their significance.
- Briefly highlight the historical development of AI and LLMs to give readers context.
2. Recent Advancements:
- Detail the key advancements in AI and LLMs over the past few years
- Discuss significant breakthroughs, such as improvements in natural language
processing, machine learning algorithms, and real-world applications.
- Include examples of how these advancements are currently being utilized across
various industries (e.g., healthcare, finance, customer service).
3. Top Generative AI Software:
- Highlight and review the best generative AI software on the market, including tools for
text generation, image synthesis, and other applications.
- Provide a balanced comparison of features, benefits, and limitations of each software.
Some prominent examples might include OpenAI's GPT-3 or GPT-4, Google’s BERT
and LaMDA, and Microsoft's Azure AI.
4. Role of Major Players:
- Explore the roles and contributions of key players in the AI landscape:
- OpenAI: Discuss its mission, major projects like GPT series, and impact on AI
research and accessibility.
- Google: Cover initiatives like DeepMind and the use of AI in Google Search, Assistant,
and cloud services.
- Microsoft: Highlight Microsoft’s integration of AI into Office 365, Azure AI services, and
partnerships with organizations like OpenAI.
- Analyze how these three giants are shaping the future of AI, their strategic goals, and
how they differ in their approach to AI development and deployment.
- Discuss any notable collaborations or rivalries between these companies and the
implications for the AI ecosystem.
5. Safety and Ethical Concerns:
- Examine the potential risks and ethical concerns associated with AI and LLMs,
including issues of bias, misinformation, privacy, and job displacement.
- Discuss the measures being taken by industry leaders and researchers to ensure the
safety and ethical use of AI.
- Highlight regulatory efforts and guidelines that aim to mitigate risks and promote
responsible AI development.
6. Future Threats and Opportunities:
- Speculate on the potential future threats posed by AI, such as the possibility of AI
surpassing human intelligence (superintelligence), cybersecurity risks, and misuse by
malicious actors.
- Conversely, discuss the transformative potential of AI, including opportunities for
solving complex global challenges, advancing scientific research, and improving quality
of life.
- Include expert opinions and forecasts to provide a balanced view of what the future
may hold.
7. Conclusion:
- Summarize the main points discussed in the blo
"""

In [81]:
jds = [jd1, jd2, jd3, jd4, jd5, jd6, jd7, jd8]

In [80]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

The top 5 applicants for JD1 are ['Aryan Tayal.pdf', 'Amit Kumar.pdf', 'Akshat Vijay.pdf', 'Anirudh Kohli.pdf', 'Dokala Avinash.pdf']
The top 5 applicants for JD2 are ['Anirudh Kohli.pdf', 'Dokala Avinash.pdf', 'Akshat Vijay.pdf', 'Amit Kumar.pdf', 'Abhishek Kumar Tiwari.pdf']
The top 5 applicants for JD3 are ['Aryan Tayal.pdf', 'Amit Kumar.pdf', 'Anirudh Kohli.pdf', 'Akshat Vijay.pdf', 'Atul Ranjan.pdf']
The top 5 applicants for JD4 are ['Dokala Avinash.pdf', 'Dibyendu Biswas.pdf', 'Anjali Raj.pdf', 'Aryan Tayal.pdf', 'Anirudh Kohli.pdf']
The top 5 applicants for JD5 are ['Anirudh Sharma.pdf', 'Aryan Tayal.pdf', 'Aryan Kumar.pdf', 'Amit Kumar.pdf', 'Aditya Krishna Das.pdf']
The top 5 applicants for JD6 are ['Aryan Kumar.pdf', 'Doraswamy Vamsi.pdf', 'Atul Ranjan.pdf', 'Amisha Sinha.pdf', 'Aryan Tayal.pdf']
The top 5 applicants for JD7 are ['Anirudh Sharma.pdf', 'Aryan Tayal.pdf', 'Amruit Sahoo.pdf', 'Ankesh Kumar.pdf', 'Aryan Kumar.pdf']
The top 5 applicants for JD8 are ['Anirudh Kohli

In [100]:
jd_list = ['Web Development Project', 'Show Me The Money', 'Hiring - Chatbot', 'Web Scraper', 'Problem Statement for Digital Marketing Professional: Website Visibility & SEO Improvement',
           'Problem Statement for UX Designers: B2B SaaS Platform for Restaurant Owners and Suppliers', 'Market Survey', 
           'Problem Statement for Content Writer and Copywriter: Blog on Advances in AI and LLMs']

In [101]:
len(jds)

8

In [115]:
best = []
model = Doc2Vec.load('cv_job_maching1.model')
# Apply to CV and JD
count = -1
for jd in jds:
    count += 1
    input_JD = preprocess_text(jd)
    similarities = []
    for resume in resumes:
        input_CV = preprocess_text(resume)
        v1 = model.infer_vector(input_CV.split())
        v2 = model.infer_vector(input_JD.split())
        similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
        similarities.append(similarity)
    res = (sorted(range(len(similarities)), key = lambda sub: similarities[sub])[-5:])
    res.reverse()
    student_names = []
    for index in res:
        student_names.append(dir_list[index])
    best.append(student_names)
    print(f"The top 5 applicants for {jd_list[count]} are {student_names}")
    

The top 5 applicants for Web Development Project are ['Aryan Tayal.pdf', 'Amit Kumar.pdf', 'Akshat Vijay.pdf', 'Anirudh Kohli.pdf', 'Dokala Avinash.pdf']
The top 5 applicants for Show Me The Money are ['Anirudh Kohli.pdf', 'Dokala Avinash.pdf', 'Akshat Vijay.pdf', 'Amit Kumar.pdf', 'Abhishek Kumar Tiwari.pdf']
The top 5 applicants for Hiring - Chatbot are ['Aryan Tayal.pdf', 'Amit Kumar.pdf', 'Anirudh Kohli.pdf', 'Akshat Vijay.pdf', 'Atul Ranjan.pdf']
The top 5 applicants for Web Scraper are ['Dokala Avinash.pdf', 'Dibyendu Biswas.pdf', 'Anjali Raj.pdf', 'Aryan Tayal.pdf', 'Anirudh Kohli.pdf']
The top 5 applicants for Problem Statement for Digital Marketing Professional: Website Visibility & SEO Improvement are ['Anirudh Sharma.pdf', 'Aryan Tayal.pdf', 'Aryan Kumar.pdf', 'Amit Kumar.pdf', 'Aditya Krishna Das.pdf']
The top 5 applicants for Problem Statement for UX Designers: B2B SaaS Platform for Restaurant Owners and Suppliers are ['Aryan Kumar.pdf', 'Doraswamy Vamsi.pdf', 'Atul Ranjan

In [116]:
best

[['Aryan Tayal.pdf',
  'Amit Kumar.pdf',
  'Akshat Vijay.pdf',
  'Anirudh Kohli.pdf',
  'Dokala Avinash.pdf'],
 ['Anirudh Kohli.pdf',
  'Dokala Avinash.pdf',
  'Akshat Vijay.pdf',
  'Amit Kumar.pdf',
  'Abhishek Kumar Tiwari.pdf'],
 ['Aryan Tayal.pdf',
  'Amit Kumar.pdf',
  'Anirudh Kohli.pdf',
  'Akshat Vijay.pdf',
  'Atul Ranjan.pdf'],
 ['Dokala Avinash.pdf',
  'Dibyendu Biswas.pdf',
  'Anjali Raj.pdf',
  'Aryan Tayal.pdf',
  'Anirudh Kohli.pdf'],
 ['Anirudh Sharma.pdf',
  'Aryan Tayal.pdf',
  'Aryan Kumar.pdf',
  'Amit Kumar.pdf',
  'Aditya Krishna Das.pdf'],
 ['Aryan Kumar.pdf',
  'Doraswamy Vamsi.pdf',
  'Atul Ranjan.pdf',
  'Amisha Sinha.pdf',
  'Aryan Tayal.pdf'],
 ['Anirudh Sharma.pdf',
  'Aryan Tayal.pdf',
  'Amruit Sahoo.pdf',
  'Ankesh Kumar.pdf',
  'Aryan Kumar.pdf'],
 ['Anirudh Kohli.pdf',
  'Aryan Tayal.pdf',
  'Anirudh Sharma.pdf',
  'Aditya Chandrikapure.pdf',
  'Aditya Krishna Das.pdf']]

In [117]:
len(best)

8

In [118]:
df = pd.DataFrame({'Projects' : jd_list,
                   'Recommended resumes' : best})

In [119]:
df

,Projects,Recommended resumes
0,Web Development Project,"[Aryan Tayal.pdf, Amit Kumar.pdf, Akshat Vijay..."
1,Show Me The Money,"[Anirudh Kohli.pdf, Dokala Avinash.pdf, Akshat..."
2,Hiring - Chatbot,"[Aryan Tayal.pdf, Amit Kumar.pdf, Anirudh Kohl..."
3,Web Scraper,"[Dokala Avinash.pdf, Dibyendu Biswas.pdf, Anja..."
4,Problem Statement for Digital Marketing Profes...,"[Anirudh Sharma.pdf, Aryan Tayal.pdf, Aryan Ku..."
5,Problem Statement for UX Designers: B2B SaaS P...,"[Aryan Kumar.pdf, Doraswamy Vamsi.pdf, Atul Ra..."
6,Market Survey,"[Anirudh Sharma.pdf, Aryan Tayal.pdf, Amruit S..."
7,Problem Statement for Content Writer and Copyw...,"[Anirudh Kohli.pdf, Aryan Tayal.pdf, Anirudh S..."


In [120]:
df.to_csv('Recommendations_cos_similarity.csv', index = False)